In [281]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import lit

# Define connection properties
url = "jdbc:sqlserver://vijaytraining.sql.azuresynapse.net:1433;database=trainingsqlpl;user=sqladminuser;password=Vijay1234;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30"
properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


file_path = "abfss://vijay@synapsdl.dfs.core.windows.net/*.ndjson"

# Read the NDJSON file into a DataFrame
df = spark.read.json(file_path)



df = df.withColumn("date_utc", lit(df["date.utc"])) \
       .withColumn("date_local", lit(df["date.local"])) \
       .drop("date")

df = df.withColumn("averagingPeriod_unit", df["averagingPeriod.unit"]) \
       .withColumn("averagingPeriod_value", df["averagingPeriod.value"]) \
       .drop("averagingPeriod")
df = df.withColumn("coordinates_lat", df["coordinates.latitude"]) \
       .withColumn("coordinates_long", df["coordinates.longitude"]) \
       .drop("coordinates")
# Show the DataFrame to verify the transformation
df=df.drop('attribution')
df.toPandas()


StatementMeta(trainingsparkpl, 0, 282, Finished, Available)

,city,country,location,mobile,parameter,sourceName,sourceType,unit,value,date_utc,date_local,averagingPeriod_unit,averagingPeriod_value,coordinates_lat,coordinates_long
0,Kabul,AF,US Diplomatic Post: Kabul,False,pm25,StateAir_Kabul,government,µg/m³,-999.000000,2021-08-15T15:30:00.000Z,2021-08-15T20:00:00+04:30,hours,1.0,34.535812,69.190514
1,Kabul,AF,US Diplomatic Post: Kabul,False,pm25,StateAir_Kabul,government,µg/m³,-999.000000,2021-08-15T16:30:00.000Z,2021-08-15T21:00:00+04:30,hours,1.0,34.535812,69.190514
2,Kabul,AF,US Diplomatic Post: Kabul,False,pm25,StateAir_Kabul,government,µg/m³,-999.000000,2021-08-15T17:30:00.000Z,2021-08-15T22:00:00+04:30,hours,1.0,34.535812,69.190514
3,Kabul,AF,US Diplomatic Post: Kabul,False,pm25,StateAir_Kabul,government,µg/m³,-999.000000,2021-08-15T18:30:00.000Z,2021-08-15T23:00:00+04:30,hours,1.0,34.535812,69.190514
4,Kabul,AF,US Diplomatic Post: Kabul,False,pm25,StateAir_Kabul,government,µg/m³,-999.000000,2021-08-15T19:30:00.000Z,2021-08-16T00:00:00+04:30,hours,1.0,34.535812,69.190514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77268,Reggio Di Calabria,IT,IT2028A,False,no2,EEA Italy,government,µg/m³,22.495998,2021-10-06T13:00:00.000Z,2021-10-06T15:00:00+02:00,hours,1.0,38.104950,15.638790
77269,Bolzano/Bozen,IT,IT0507A,False,no2,EEA Italy,government,µg/m³,19.870000,2021-10-06T14:00:00.000Z,2021-10-06T16:00:00+02:00,hours,1.0,46.715280,11.654440
77270,Napoli,IT,IT1497A,False,no2,EEA Italy,government,µg/m³,7.550000,2021-10-06T13:00:00.000Z,2021-10-06T15:00:00+02:00,hours,1.0,40.862778,14.254444
77271,Latina,IT,IT1184A,False,no2,EEA Italy,government,µg/m³,23.000000,2021-10-06T14:00:00.000Z,2021-10-06T16:00:00+02:00,hours,1.0,41.531389,12.946111


In [282]:


# Define the target table name in your dedicated SQL pool
table_name = "air_quality"
#df = df.withColumn("city", df["city"].cast("string"))
# Write the DataFrame to the dedicated SQL pool
df.write.jdbc(url=url, table=table_name, mode="append", properties=properties)


StatementMeta(trainingsparkpl, 0, 283, Finished, Cancelled)

In [ ]:
path = "abfss://vijay@synapsdl.dfs.core.windows.net/countries.csv"

df = spark.read.option("header", True).csv(path)
df.show()
df.write.jdbc(url=url, table='country', mode="append", properties=properties)

StatementMeta(, , , Cancelled, )

In [ ]:


df_air = spark.read.jdbc(url=url, table="air_quality", properties=properties)
df_country = spark.read.jdbc(url=url, table="country", properties=properties)


StatementMeta(, , , Cancelled, )

In [ ]:
# Assuming we're filtering df1 based on values in 'id' column of df2

# Collect unique 'id' values from df2 into a list
country_to_filter = df_country.select("country_code").distinct().rdd.flatMap(lambda x: x).collect()

# Filter df1 where 'id' is in the collected list
filtered_df_air = df_air.filter(df_air["country"].isin(country_to_filter))





StatementMeta(, , , Cancelled, )

In [ ]:
# Filter for required parameters
filtered_df_air.select('parameter').distinct().show()
param_list=['PM25', 'PM10', 'O3', 'NO2', 'CO']
filtered_df_air1 = filtered_df_air.filter(filtered_df_air["parameter"].isin(param_list))

StatementMeta(, , , Cancelled, )

In [ ]:
filtered_df_air1.select('parameter').distinct().show()
filtered_df_air2.dtypes

StatementMeta(, , , Cancelled, )

In [ ]:
filtered_df_air1 = filtered_df_air.select('city','country','parameter','unit','value','date_utc')


StatementMeta(, , , Cancelled, )

In [ ]:
filtered_df_air2=filtered_df_air1.filter(filtered_df_air1['value'] > 0)

StatementMeta(, , , Cancelled, )

In [ ]:
filtered_df_air2.filter(filtered_df_air1['value'] < 0).show()

StatementMeta(, , , Cancelled, )

In [ ]:
filtered_df_air2.select('date_utc').show(1)

StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import col, to_timestamp, avg, expr
from pyspark.sql.window import Window

# Ensure 'date_utc' is a datetime type
#filtered_df_air2 = filtered_df_air2.withColumn("date_utc", to_timestamp(col("date_utc"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z''"))
filtered_df_air2 = filtered_df_air2.withColumn('date_utc', filtered_df_air2.date_utc.cast('timestamp'))
filtered_df_air2 = filtered_df_air2.withColumn('value', filtered_df_air2.value.cast('int'))

StatementMeta(, , , Cancelled, )

In [ ]:
# Define the window specification for a 24-hour rolling window
#window_spec = Window.partitionBy("city", "parameter").orderBy("date_utc").rangeBetween(-86400, 0)



StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import expr
from pyspark.sql import functions as F
days = lambda i: i * 86400
window_spec = Window \
    .partitionBy("city", "parameter") \
    .orderBy(F.col("date_utc").cast('long')) \
    .rangeBetween(-days(1), 0)


    #orderBy(F.col("timestampGMT").cast('long'))

StatementMeta(, , , Cancelled, )

In [290]:
# Calculate the rolling average
df_rolling_avg = filtered_df_air2.withColumn("24hr_rolling_avg", avg("value").over(window_spec))



StatementMeta(trainingsparkpl, 0, 291, Finished, Available)

In [294]:
# Writing Rolling Avg to DB


df_rolling_avg.write.jdbc(url=url, table='air_roll_avg', mode="append", properties=properties)

StatementMeta(trainingsparkpl, 0, 295, Finished, Cancelled)

In [303]:
df_rolling_avg.show()

StatementMeta(trainingsparkpl, 0, 304, Finished, Available)

+--------+-------+---------+-----+-----+-------------------+----------------+
|    city|country|parameter| unit|value|           date_utc|24hr_rolling_avg|
+--------+-------+---------+-----+-----+-------------------+----------------+
|Aberdeen|     GB|      no2|µg/m³|   39|2021-10-06 13:00:00|            39.5|
|Aberdeen|     GB|      no2|µg/m³|   40|2021-10-06 13:00:00|            39.5|
|Aberdeen|     GB|      no2|µg/m³|   37|2021-10-06 14:00:00|            42.5|
|Aberdeen|     GB|      no2|µg/m³|   54|2021-10-06 14:00:00|            42.5|
|Aberdeen|     GB|      no2|µg/m³|   53|2021-10-06 16:00:00|            43.0|
|Aberdeen|     GB|      no2|µg/m³|   25|2021-10-06 16:00:00|            43.0|
|Aberdeen|     GB|      no2|µg/m³|   53|2021-10-06 16:00:00|            43.0|
|Aberdeen|     GB|      no2|µg/m³|   24|2021-10-06 17:00:00|          40.625|
|Aberdeen|     GB|      no2|µg/m³|   34|2021-10-06 18:00:00|            37.5|
|Aberdeen|     GB|      no2|µg/m³|   16|2021-10-06 18:00:00|    

In [314]:
# AQI calculation function
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType
def calculate_aqi(pm_obs, breakpoint_low, breakpoint_high, aqi_low, aqi_high):
    aqi = ((aqi_high - aqi_low) / (breakpoint_high - breakpoint_low)) * (pm_obs - breakpoint_low) + aqi_low
    return aqi

# Register the function as a UDF
calculate_aqi_udf = udf(calculate_aqi, DoubleType())


# Define the AQI breakpoints and AQI values for PM2.5
breakpoint_low_PM25 = 12.1
breakpoint_high_PM25 = 35.4
aqi_low_PM25 = 51
aqi_high_PM25 = 100


# Define the AQI breakpoints and AQI values for PM10
breakpoint_low_PM10 = 55.0
breakpoint_high_PM10 = 154.0
aqi_low_PM10= 51
aqi_high_PM10 = 100



StatementMeta(trainingsparkpl, 0, 315, Finished, Available)

In [311]:
df_rolling_avg_PM25=df_rolling_avg.filter(df_rolling_avg['parameter'] == 'PM25')
df_rolling_avg_PM10=df_rolling_avg.filter(df_rolling_avg['parameter'] == 'PM10')

StatementMeta(trainingsparkpl, 0, 312, Finished, Available)

In [312]:
# Apply the UDF to calculate the AQI for PM2.5
df_aqi_pm25 = df_rolling_avg_PM25.withColumn("aqi_pm25", calculate_aqi_udf(
    col("24hr_rolling_avg"), 
    lit(breakpoint_low_PM25), 
    lit(breakpoint_high_PM25), 
    lit(aqi_low_PM25), 
    lit(aqi_high_PM25)
))

StatementMeta(trainingsparkpl, 0, 313, Finished, Available)

In [317]:
df_aqi_pm25.show()

StatementMeta(trainingsparkpl, 0, 318, Finished, Available)

+-----+-------+---------+-----+-----+-------------------+------------------+------------------+
| city|country|parameter| unit|value|           date_utc|  24hr_rolling_avg|          aqi_pm25|
+-----+-------+---------+-----+-----+-------------------+------------------+------------------+
|  Ain|     FR|     pm25|µg/m³|    3|2021-10-06 09:00:00|               3.0|31.862660944206006|
|  Ain|     FR|     pm25|µg/m³|    4|2021-10-06 10:00:00|               3.5| 32.91416309012875|
|  Ain|     FR|     pm25|µg/m³|    4|2021-10-06 11:00:00|3.6666666666666665|33.264663805436335|
|  Ain|     FR|     pm25|µg/m³|    3|2021-10-06 12:00:00|               3.5| 32.91416309012875|
|  Ain|     FR|     pm25|µg/m³|    2|2021-10-06 13:00:00|               3.0|31.862660944206006|
|  Ain|     FR|     pm25|µg/m³|    2|2021-10-06 13:00:00|               3.0|31.862660944206006|
|  Ain|     FR|     pm25|µg/m³|    2|2021-10-06 14:00:00|              2.75|31.336909871244632|
|  Ain|     FR|     pm25|µg/m³|    2|202

In [318]:
df_aqi_pm10 = df_rolling_avg_PM10.withColumn("aqi_pm10", calculate_aqi_udf(
    col("24hr_rolling_avg"), 
    lit(breakpoint_low_PM10), 
    lit(breakpoint_high_PM10), 
    lit(aqi_low_PM10), 
    lit(aqi_high_PM10)
))

StatementMeta(trainingsparkpl, 0, 319, Finished, Available)

In [319]:
df_aqi_pm10.show()

StatementMeta(trainingsparkpl, 0, 320, Finished, Available)

+---------------+-------+---------+-----+-----+-------------------+-----------------+------------------+
|           city|country|parameter| unit|value|           date_utc| 24hr_rolling_avg|          aqi_pm10|
+---------------+-------+---------+-----+-----+-------------------+-----------------+------------------+
|            Ain|     FR|     pm10|µg/m³|    8|2021-10-06 09:00:00|              7.0|27.242424242424242|
|            Ain|     FR|     pm10|µg/m³|    6|2021-10-06 09:00:00|              7.0|27.242424242424242|
|            Ain|     FR|     pm10|µg/m³|   10|2021-10-06 10:00:00|              8.0|27.737373737373737|
|            Ain|     FR|     pm10|µg/m³|    5|2021-10-06 11:00:00|              7.0|27.242424242424242|
|            Ain|     FR|     pm10|µg/m³|    8|2021-10-06 11:00:00|              7.0|27.242424242424242|
|            Ain|     FR|     pm10|µg/m³|    5|2021-10-06 11:00:00|              7.0|27.242424242424242|
|            Ain|     FR|     pm10|µg/m³|    8|2021-10-

In [320]:
#Loading the AQI tables into the DB.
df_aqi_pm10.write.jdbc(url=url, table='air_aqi_pm10', mode="append", properties=properties)
df_aqi_pm25.write.jdbc(url=url, table='air_aqi_pm25', mode="append", properties=properties)

StatementMeta(trainingsparkpl, 0, 321, Finished, Available)